# **Data** **Loading**

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

import os


Making a connection to google drive in order to get data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Data Preprocessing**

In [ ]:
driveCyclesPath = "/content/drive/MyDrive/degreecycle/20 degrees"
data = None

# Access each drive cycle
for driveCycle in os.listdir(driveCyclesPath):

  if os.path.isdir(driveCyclesPath+"/"+driveCycle):

# Access each city folder in the drive cycles
      for city in os.listdir(driveCyclesPath+"/"+driveCycle):

# Access weather files in each city 
        for weather in ["Summer","Winter"]:

# Read the drive cycle file from each season folder and setting up the dataset with the appropriate variables in order to setup a dataset.
          df = pd.read_csv(driveCyclesPath+"/"+driveCycle+"/"+city+"/"+weather+"/Drive_cycle-v2.csv")
          dfT = df.transpose()
          dfT.reset_index(inplace=True)
          dfT.columns = dfT.iloc[0].values
          newCitydf = dfT.iloc[1:]

# Trying to concatenate all the files by adding drive cycle, city and weather as their respective features.
          newCitydf = newCitydf.assign(driveCycle=driveCycle)
          newCitydf = newCitydf.assign(city=city)
          newCitydf = newCitydf.assign(weather=weather)

          try:
            data = pd.concat([data, newCitydf], ignore_index=True)
          except:
            data = newCitydf


data.head()


,Degrees,Time,Speed,Energycons,EProp,E_Aux,HVAC,Power,Regen,SoC,driveCycle,city,weather
0,0,0.0,0.0,NaN,0.0,5.35,4.2501,9.6001,0.0,100.0,UDDS,Seattle,Summer
1,0.1,0.000039,0.0,inf,0.0,5.35,4.2501,9.6001,0.0,100.0,UDDS,Seattle,Summer
2,0.2,0.55586,0.0,inf,0.0,5.35,4.2501,9.6001,0.0,100.0,UDDS,Seattle,Summer
3,0.3,5.5,0.0,inf,0.0,5.35,4.2501,9.6001,0.0,99.995,UDDS,Seattle,Summer
4,0.4,9.7022,0.0,inf,0.0,5.35,4.2501,9.6001,0.0,99.991,UDDS,Seattle,Summer


In [ ]:
data.to_csv("revoltData.csv")

In [ ]:
data = pd.read_csv("revoltData.csv")

Now to divide the file into multiple files based on drive cycle, we use groupby method.

In [ ]:
groupeddata = data.groupby('driveCycle')

In [ ]:
UDDS_data = groupeddata.get_group(data['driveCycle'].unique()[0])
Manh_data = groupeddata.get_group(data['driveCycle'].unique()[1])
NYB_data = groupeddata.get_group(data['driveCycle'].unique()[2])
Oran_data = groupeddata.get_group(data['driveCycle'].unique()[3])
NYC_data = groupeddata.get_group(data['driveCycle'].unique()[4])
CBD_data = groupeddata.get_group(data['driveCycle'].unique()[5])



Checking if all the cities and weather exists in the files. We don't have to check for all the files as we have applied the code in the same manner. Checking for two files will help us know if the files have different shape with required amount of rows and columns.

In [ ]:
print(Manh_data.nunique())

Unnamed: 0    6917
Degrees        329
Time          5528
Speed         3053
Energycons    5358
EProp         3330
E_Aux            2
HVAC            26
Power         4495
Regen          708
SoC           4034
driveCycle       1
city            10
weather          2
dtype: int64


In [ ]:
print(NYC_data.nunique())

Unnamed: 0    8405
Degrees        397
Time          4700
Speed         5172
Energycons    6960
EProp         5429
E_Aux            2
HVAC            26
Power         6224
Regen         2038
SoC           4299
driveCycle       1
city            10
weather          2
dtype: int64


In [ ]:
UDDS_data.to_csv("UDDS")
Manh_data.to_csv("Manhattan")
NYB_data.to_csv("NewYorkBus")
Oran_data.to_csv("OrangeCountry")
NYC_data.to_csv("NewYorkComp")
CBD_data.to_csv("CBD")
